In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 53.3 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from string import digits
from collections import Counter
from pyvi import ViTokenizer
from gensim.models.word2vec import Word2Vec
from keras.utils import to_categorical
%matplotlib inline

In [ ]:
#!cp -r "/content/drive/MyDrive/Tài liệu HCMUT/NLP Modern/W2V+[LSTM, CRNN] Sentiment Analysis" /content

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/my-project/LabNLP/Lab6-W2V+[LSTM, CRNN] Sentiment Analysis"

/content/drive/MyDrive/Colab Notebooks/my-project/LabNLP/Lab6-W2V+[LSTM, CRNN] Sentiment Analysis


In [ ]:
data_train = pd.read_csv("vlsp_sentiment_train.csv", sep='\t')
data_train.columns =['Class', 'Data']
data_test = pd.read_csv("vlsp_sentiment_test.csv", sep='\t')
data_test.columns =['Class', 'Data']

In [ ]:
print(data_train.shape)
print(data_test.shape)
print(type(data_train))

(5100, 2)
(1050, 2)
<class 'pandas.core.frame.DataFrame'>


In [ ]:
# part_test = data_test[:400]
# #data_train += part_test
# data_test = data_test[400:]
# print(part_test.shape)

In [ ]:
# data_train = pd.concat([data_train,part_test])
# print(data_train.shape)

In [ ]:
labels = data_train.iloc[:, 0].values
reviews = data_train.iloc[:, 1].values

In [ ]:
encoded_labels = []

for label in labels:
    if label == -1:
        encoded_labels.append([1,0,0])
    elif label == 0:
        encoded_labels.append([0,1,0])
    else:
        encoded_labels.append([0,0,1])

encoded_labels = np.array(encoded_labels)

In [ ]:
reviews_processed = []
unlabeled_processed = []
for review in reviews:
    review_cool_one = ''.join([char for char in review if char not in digits])
    reviews_processed.append(review_cool_one)

In [ ]:
#Use PyVi for Vietnamese word tokenizer
word_reviews = []
all_words = []
for review in reviews_processed:
    review = ViTokenizer.tokenize(review.lower())
    word_reviews.append(review.split())


In [ ]:
EMBEDDING_DIM = 400 # how big is each word vector
MAX_VOCAB_SIZE = 10000 # how many unique words to use (i.e num rows in embedding vector)
MAX_SEQUENCE_LENGTH = 300 # max number of words in a comment to use

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, lower=True, char_level=False)
tokenizer.fit_on_texts(word_reviews)
sequences_train = tokenizer.texts_to_sequences(word_reviews)
word_index = tokenizer.word_index


In [ ]:
data = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)
labels = encoded_labels

In [ ]:
print('Shape of X train and X validation tensor:',data.shape)
print('Shape of label train and validation tensor:', labels.shape)

Shape of X train and X validation tensor: (5100, 300)
Shape of label train and validation tensor: (5100, 3)


In [ ]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Colab Notebooks/my-project/LabNLP/Lab5-CNN+W2V_Sentiment Analysis/vi-model-CBOW.bin', binary=True)


vocabulary_size=min(len(word_index)+1,MAX_VOCAB_SIZE)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
print("vocab size", vocabulary_size)
for word, i in word_index.items():
    if i>=MAX_VOCAB_SIZE:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

vocab size 7919


In [ ]:
from keras.models import Model
from keras.layers import *
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
sequence_length = data.shape[1]
filter_sizes = [1,2,3,4]
num_filters = 512
drop = 0.46

inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)

################## LSTM ONLY ###############################
#reshape = Reshape((sequence_length,EMBEDDING_DIM))(embedding)

################# SINGLE LSTM ####################
#lstm_0 = LSTM(512)(reshape)

# YOU W# ANNA ADD MORE LSTM LAYERS? UNCOMMENT THIS #
# lstm_2 = LSTM(128, return_sequences=True)(reshape)
# lstm_1 = LSTM(256, return_sequences=True)(lstm_2)
# lstm_3 = LSTM(512, return_sequences=True)(lstm_1)
#lstm_0 = LSTM(256)(lstm_3)

############################################################

################## LSTM -> FC -> CNN ########################
# reshape = Reshape((sequence_length,EMBEDDING_DIM))(embedding)

# lstm_3 = LSTM(64, dropout=0.4)(reshape)
# lstm_0 = LSTM(64, dropout=0.4)(reshape)
# lstm_1 = LSTM(64, dropout=0.4)(reshape)
# lstm_2 = LSTM(64, dropout=0.4)(reshape)
# lstm_4 = LSTM(64, dropout=0.4)(reshape)
# lstm_5 = LSTM(64, dropout=0.4)(reshape)
# lstm_6 = LSTM(64, dropout=0.4)(reshape)

# concat_lstm = concatenate([lstm_0,lstm_1,lstm_2,lstm_3,lstm_4,lstm_5,lstm_6])

# reshape_emb = Reshape((448, 1))(concat_lstm)

# reshape_fc = Dense(448)(reshape_emb)

# conv_0 = Conv1D(num_filters, (filter_sizes[0], ),padding="same",activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape_fc)
# conv_1 = Conv1D(num_filters, (filter_sizes[1], ),padding="same",activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape_fc)
# conv_2 = Conv1D(num_filters, (filter_sizes[2], ),padding="same",activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape_fc)
# #conv_3 = Conv1D(num_filters, (filter_sizes[2], ),padding="same",activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape_fc)

# conv_0 = MaxPool1D(300)(conv_0)
# conv_1 = MaxPool1D(300)(conv_1)
# conv_2 = MaxPool1D(300)(conv_2)
# #conv_3 = MaxPool1D(300)(conv_3)

# concat = concatenate([conv_0, conv_1, conv_2], axis=1)

# flatten = Flatten()(concat)
# #dropout = Dropout(drop)(flatten)
# output = Dense(units=3, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(flatten)

# output = Flatten()(output)

###########################################################


################## CRNN ####################################
reshape = Reshape((sequence_length,EMBEDDING_DIM))(embedding)


conv_0 = Conv1D(num_filters, (filter_sizes[0], ),padding="same",activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv1D(num_filters, (filter_sizes[1], ),padding="same",activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv1D(num_filters, (filter_sizes[2], ),padding="same",activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_3 = Conv1D(num_filters, (filter_sizes[2], ),padding="same",activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

# concat_0 = concatenate([conv_0, conv_1, conv_2])

# conv_conv_0 = Conv1D(num_filters, (filter_sizes[0], ),padding="same",activation='relu',kernel_regularizer=regularizers.l2(0.01))(concat_0)
# conv_conv_1 = Conv1D(num_filters, (filter_sizes[1], ),padding="same",activation='relu',kernel_regularizer=regularizers.l2(0.01))(concat_0)
# conv_conv_2 = Conv1D(num_filters, (filter_sizes[2], ),padding="same",activation='relu',kernel_regularizer=regularizers.l2(0.01))(concat_0)

conv_0 = MaxPool1D(300)(conv_0)
conv_1 = MaxPool1D(300)(conv_1)
conv_2 = MaxPool1D(300)(conv_2)
conv_3 = MaxPool1D(300)(conv_3)
# Reshape output to match RNN dimension
conv_0 = Reshape((-1, num_filters))(conv_0)
conv_1 = Reshape((-1, num_filters))(conv_1)
conv_2 = Reshape((-1, num_filters))(conv_2)
conv_3 = Reshape((-1, num_filters))(conv_3)

concat = concatenate([conv_0, conv_1, conv_2,conv_3])
#concat = Flatten()(concat)
#lstm_1 = LSTM(512, return_sequences=True, dropout=0.2)(concat)
lstm_0 = LSTM(256,dropout=0.38)(concat)

#YOU WANNA ADD MORE LSTM LAYERS? UNCOMMENT THIS #
# lstm_2 = LSTM(1024, return_sequences=True)(concat)
# lstm_1 = LSTM(512, return_sequences=True)(lstm_2)
# lstm_0 = LSTM(256)(lstm_1)

############################################################

################### without LSTM + CNN ###############################
#dropout = Dropout(drop)(concat)
dropout = Dropout(drop)(lstm_0)
output = Dense(units=3, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)
output = Flatten()(output)

# this creates a model that includes
model = Model(inputs, output)


adam = tf.keras.optimizers.legacy.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 300)]                0         []                            
                                                                                                  
 embedding_3 (Embedding)     (None, 300, 400)             3167600   ['input_4[0][0]']             
                                                                                                  
 reshape_15 (Reshape)        (None, 300, 400)             0         ['embedding_3[0][0]']         
                                                                                                  
 conv1d_12 (Conv1D)          (None, 300, 512)             205312    ['reshape_15[0][0]']          
                                                                                            

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
### IF YOU HAVE MODEL WEIGHT AND WANNA LOAD IT
#model.load_weights("model.h5")

In [ ]:
#define callbacks
# early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
# callbacks_list = [early_stopping]

# model.fit(data, labels, validation_split=0.2,
#           epochs=1, batch_size=128, callbacks=callbacks_list, shuffle=True)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]

In [ ]:
labels_test = data_test.iloc[:, 0].values
reviews_test = data_test.iloc[:, 1].values

In [ ]:
encoded_labels_test = []

for label_test in labels_test:
    if label_test == -1:
        encoded_labels_test.append([1,0,0])
    elif label_test == 0:
        encoded_labels_test.append([0,1,0])
    else:
        encoded_labels_test.append([0,0,1])

encoded_labels_test = np.array(encoded_labels_test)

In [ ]:
reviews_processed_test = []
unlabeled_processed_test = []
for review_test in reviews_test:
    review_cool_one = ''.join([char for char in review_test if char not in digits])
    reviews_processed_test.append(review_cool_one)

In [ ]:
#Use PyVi for Vietnamese word tokenizer
word_reviews_test = []
all_words = []
for review_test in reviews_processed_test:
    review_test = ViTokenizer.tokenize(review_test.lower())
    word_reviews_test.append(review_test.split())

In [ ]:
sequences_test = tokenizer.texts_to_sequences(word_reviews_test)
data_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
labels_test = encoded_labels_test

In [ ]:
print('Shape of X train and X validation tensor:',data_test.shape)
print('Shape of label train and validation tensor:', labels_test.shape)

Shape of X train and X validation tensor: (1050, 300)
Shape of label train and validation tensor: (1050, 3)


In [ ]:
for i in range(1,12):
  model.fit(data, labels, validation_split=0.1,
          epochs=1, batch_size=128, callbacks=callbacks_list, shuffle=True)
  score = model.evaluate(data_test, labels_test)



33/33 [==============================] - 0s 8ms/step - loss: 1.4240 - accuracy: 0.6771


In [ ]:
#score = model.evaluate(data_test, labels_test)

In [ ]:
print("%s: %.2f" % (model.metrics_names[0], score[0]))
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))


loss: 1.22
accuracy: 68.67%


In [ ]:
%cd /content

/content


In [ ]:
model.save_weights("model.h5")

In [ ]:
!cp model.h5 /content/drive/MyDrive